In [61]:
%defaultDatasource jdbc:h2:mem:db

In [62]:
DROP TABLE IF EXISTS RookMate;
DROP TABLE IF EXISTS KnightMate;
DROP TABLE IF EXISTS QueenMate;
DROP TABLE IF EXISTS BishopMate;
DROP TABLE IF EXISTS PawnMate;
DROP TABLE IF EXISTS TodosMates;
DROP TABLE IF EXISTS TakesPromotion;
DROP TABLE IF EXISTS Promotion;
DROP TABLE IF EXISTS PieceMove;
DROP TABLE IF EXISTS PawnTakes;
DROP TABLE IF EXISTS PawnMove;
DROP TABLE IF EXISTS Castle;
DROP TABLE IF EXISTS Takes;
DROP TABLE IF EXISTS Substrings;
DROP TABLE IF EXISTS Mated;
DROP TABLE IF EXISTS RoqueG;
DROP TABLE IF EXISTS RoqueC;
DROP TABLE IF EXISTS AllG;
DROP TABLE IF EXISTS Winner;
DROP TABLE IF EXISTS MaxOpenings;

# Criação da Tabela de Players

In [63]:
DROP Table IF EXISTS Players;

CREATE TABLE Players (
    Username VARCHAR(25) NOT NULL,
    Title VARCHAR(3),
    Bullet INTEGER,
    Blitz INTEGER,
    Rapid INTEGER,
    SiteName VARCHAR(10) NOT NULL,
    PRIMARY KEY(Username, SiteName)
) AS SELECT
    username,
    title,
    bullet_rating,
    blitz_rating,
    rapid_rating,
    site_name
    FROM CSVREAD('../data/processed/all_player_list.csv')

In [64]:
SELECT * FROM Players

# Criação da Tabela de Jogos

In [65]:
DROP Table IF EXISTS Games;

CREATE TABLE Games (
    Id VARCHAR(40) NOT NULL,
    Ranked BOOLEAN NOT NULL,
    Time_control VARCHAR(20) NOT NULL,
    Result VARCHAR(20) NOT NULL,
    Winner VARCHAR(25),
    White VARCHAR(25) NOT NULL,
    Black VARCHAR(25) NOT NULL,
    Moves VARCHAR(9999),
    Openings VARCHAR(95),
    SiteName VARCHAR(10) NOT NULL,
    PRIMARY KEY(Id),
) AS SELECT
    id,
    ranked,
    time_control,
    status,
    winner,
    white,
    black,
    moves,
    opening,
    site_name
    FROM CSVREAD('../data/processed/all_games.csv')

In [66]:
SELECT * FROM Games G

# Criação da Tabela de Openings

In [67]:
DROP Table IF EXISTS Openings;

CREATE TABLE Openings (
    Eco VARCHAR(3),
    Opening VARCHAR(95),
    Pgn VARCHAR(210),
    PRIMARY KEY (Pgn)
) AS SELECT
    eco,
    name,
    pgn
    FROM CSVREAD('../data/processed/openings.csv')

In [68]:
SELECT * FROM Openings

# Análises realizadas no Database

In [69]:
SELECT * FROM Games G
    LEFT JOIN Players PW
    ON G.white = PW.Username AND G.SiteName = PW.SiteName
    LEFT JOIN Players PB
    ON G.black = PB.Username AND G.SiteName = PB.SiteName
    

In [70]:
DROP TABLE IF EXISTS Winner;

CREATE VIEW Winner AS
SELECT P.Username, COUNT(*) Ganhadores
    FROM Games G, Players P
    WHERE G.Winner = P.Username AND G.SiteName = 'lichess'
    GROUP BY P.Username;

SELECT P.Username, P.Title, P.SiteName, W.Ganhadores
FROM Winner W, Players P
WHERE W.Username = P.Username AND W.Ganhadores = (SELECT MAX(Winner.Ganhadores) FROM Winner)

In [71]:
DROP TABLE IF EXISTS MaxOpenings;

CREATE VIEW MaxOpenings AS
SELECT G.Openings, COUNT(*) Aberturas
    FROM Games G, Players P
    WHERE P.Username = 'kirillgenius' AND (P.Username = G.White OR P.Username = G.Black)
    GROUP BY G.Openings
    ORDER BY COUNT(*) DESC;

SELECT * FROM MaxOpenings

In [72]:
SELECT G.result, COUNT (*) OCORRENCIAS
FROM Games G
GROUP BY G.result
ORDER BY OCORRENCIAS DESC

In [73]:
SELECT X.winner, COUNT (*) WINS
FROM (SELECT G.result, G.winner,G.white,G.black
FROM Games G
WHERE G.winner = G.white OR G.winner = G.Black) X
GROUP BY X.winner
ORDER BY WINS DESC;

In [74]:
DROP TABLE IF EXISTS RoqueG;
DROP TABLE IF EXISTS RoqueC;
DROP TABLE IF EXISTS AllG;

CREATE VIEW AllG AS
SELECT COUNT(*) Total
FROM GAMES G;

CREATE VIEW RoqueG AS
SELECT COUNT(*) RoqueGrande
FROM GAMES G 
WHERE MOVES like '%O-O-O%';

CREATE VIEW RoqueC AS
SELECT COUNT(*) RoqueCurto
FROM GAMES G 
WHERE MOVES like '% O-O %' OR MOVES like '% O-O+%' OR MOVES like '% O-O#%';

SELECT G.Total, RG.RoqueGrande, RC.RoqueCurto, (RG.RoqueGrande + RC.RoqueCurto) AS Roques
FROM RoqueG RG, RoqueC RC, AllG G


In [105]:
DROP TABLE IF EXISTS Teste;
DROP TABLE IF EXISTS KingMate;
DROP TABLE IF EXISTS PecasMates;
DROP TABLE IF EXISTS RookMate;
DROP TABLE IF EXISTS KnightMate;
DROP TABLE IF EXISTS QueenMate;
DROP TABLE IF EXISTS BishopMate;
DROP TABLE IF EXISTS PawnMate;
DROP TABLE IF EXISTS TodosMates;
DROP TABLE IF EXISTS TakesPromotion; --Check
DROP TABLE IF EXISTS Promotion; --Check
DROP TABLE IF EXISTS PieceMove; --Check
DROP TABLE IF EXISTS PawnTakes; --Check
DROP TABLE IF EXISTS PawnMove; --Check
DROP TABLE IF EXISTS Castle; --Check
DROP TABLE IF EXISTS Takes; --Check
DROP TABLE IF EXISTS Substrings;
DROP TABLE IF EXISTS Mated;

CREATE VIEW Mated AS
    SELECT LEFT(RIGHT(G.MOVES,11),7) As MOVES
    FROM GAMES G
    WHERE MOVES like '%#%' AND RESULT like '%mate%';

CREATE VIEW Substrings AS
    SELECT SUBSTRING(M.MOVES, CHARINDEX(' ', M.MOVES)+1, LENGTH(M.MOVES)) AS Movement
    FROM Mated M;

CREATE VIEW TakesPromotion AS
    SELECT S.Movement
    FROM Substrings S
    WHERE LENGTH(S.Movement) = 7 AND S.Movement like '%=%' AND S.Movement like '%x%';

CREATE VIEW PawnMove AS
    SELECT S.Movement
    FROM Substrings S
    WHERE LENGTH(S.Movement) = 3 
    OR (LENGTH(S.Movement) = 6 AND SUBSTRING(S.Movement, 1, 4) like '% %');

CREATE VIEW PieceMove AS
    SELECT S.Movement
    FROM Substrings S
    WHERE (LENGTH(S.Movement) = 4 AND LEFT(S.Movement,1) NOT like '%O%')
    OR (LENGTH(S.Movement) = 5 AND (SUBSTRING(S.Movement, 1, 2) NOT like '%x%' AND SUBSTRING(S.Movement, 1, 3) NOT like '%=%'))
    OR (LENGTH(S.Movement) = 6 AND (SUBSTRING(S.Movement, 1, 3) NOT like '%x%' AND SUBSTRING(S.Movement, 1, 4) NOT like '%=%' 
    AND LEFT(S.Movement, 1) NOT like '%O%' AND SUBSTRING(S.Movement, 1, 4) NOT like '% %'));

CREATE VIEW Promotion AS
    SELECT S.Movement
    FROM Substrings S
    WHERE LENGTH(S.Movement) = 5 AND S.Movement like '%=%';

CREATE VIEW PawnTakes AS
    SELECT S.Movement
    FROM Substrings S
    WHERE LENGTH(S.Movement) = 5 AND S.Movement like '%x%' 
        AND (UPPER(LEFT(S.Movement,1)) != LEFT(S.Movement,1));

CREATE VIEW Castle AS
    SELECT S.Movement
    FROM Substrings S
    WHERE S.Movement like '%O-O%';

CREATE VIEW Takes AS
    SELECT S.Movement
    FROM Substrings S
    WHERE (LENGTH(S.Movement) = 5 AND S.Movement like '%x%' AND (UPPER(LEFT(S.Movement,1)) = LEFT(S.Movement,1))) 
        OR (LENGTH(S.Movement) = 6 AND S.Movement like '%x%' AND (UPPER(LEFT(S.Movement,1)) = LEFT(S.Movement,1))) 
        OR (LENGTH(S.Movement) = 7 AND S.Movement like '%x%' AND (UPPER(LEFT(S.Movement,1)) = LEFT(S.Movement,1)));

In [106]:
DROP TABLE IF EXISTS TodosMates;

CREATE VIEW TodosMates AS
    SELECT 'Takes + Promotion' TABELA,  COUNT(*) Total FROM TakesPromotion 
    UNION SELECT 'Promotion' TABELA,  COUNT(*) Total FROM Promotion
    UNION SELECT 'Piece Move' TABELA,  COUNT(*) Total FROM PieceMove
    UNION SELECT 'Pawn Takes' TABELA,  COUNT(*) Total FROM PawnTakes
    UNION SELECT 'Pawn Move' TABELA,  COUNT(*) Total FROM PawnMove
    UNION SELECT 'Roque' TABELA,  COUNT(*) Total FROM Castle
    UNION SELECT 'Piece Takes' TABELA,  COUNT(*) Total FROM Takes
    UNION SELECT 'Mated' TABELA,  COUNT(*) Total FROM Mated;

SELECT TM1.TABELA, ((CAST(TM1.Total AS FLOAT)/TM.Total)*100) Porcentagem
FROM TodosMates TM, TodosMates TM1
WHERE TM.TABELA = 'Mated' AND TM1.TABELA != 'Mated'
ORDER BY Porcentagem DESC;

In [108]:
DROP TABLE IF EXISTS PecasMates;
DROP TABLE IF EXISTS RookMate;
DROP TABLE IF EXISTS KnightMate;
DROP TABLE IF EXISTS QueenMate;
DROP TABLE IF EXISTS BishopMate;
DROP TABLE IF EXISTS PawnMate;
DROP TABLE IF EXISTS KingMate;

Create View KingMate AS
SELECT PieceMove.Movement FROM PieceMove WHERE PieceMove.MOVEMENT like '%K%'
UNION ALL SELECT Takes.Movement FROM Takes WHERE Takes.MOVEMENT like '%K%'; 

Create View Rookmate AS
SELECT PieceMove.Movement FROM PieceMove WHERE PieceMove.MOVEMENT like '%R%'
UNION ALL SELECT Takes.Movement FROM Takes WHERE Takes.MOVEMENT like '%R%' 
UNION ALL SELECT TakesPromotion.Movement FROM TakesPromotion WHERE TakesPromotion.MOVEMENT like '%R%' 
UNION ALL SELECT Promotion.Movement FROM Promotion WHERE Promotion.MOVEMENT like '%R%' 
UNION ALL SELECT Castle.Movement FROM Castle; 

CREATE VIEW KnightMate AS
SELECT PieceMove.Movement FROM PieceMove WHERE PieceMove.MOVEMENT like '%N%'
UNION ALL SELECT Takes.Movement FROM Takes WHERE Takes.MOVEMENT like '%N%'
UNION ALL SELECT TakesPromotion.Movement FROM TakesPromotion WHERE TakesPromotion.MOVEMENT like '%N%'
UNION ALL SELECT Promotion.Movement FROM Promotion WHERE Promotion.MOVEMENT like '%N%';

CREATE VIEW QueenMate AS
SELECT PieceMove.Movement FROM PieceMove WHERE PieceMove.MOVEMENT like '%Q%'
UNION ALL SELECT Takes.Movement FROM Takes WHERE Takes.MOVEMENT like '%Q%'
UNION ALL SELECT TakesPromotion.Movement FROM TakesPromotion WHERE TakesPromotion.MOVEMENT like '%Q%'
UNION ALL SELECT Promotion.Movement FROM Promotion WHERE Promotion.MOVEMENT like '%Q%';

CREATE VIEW BishopMate AS
SELECT PieceMove.Movement FROM PieceMove WHERE PieceMove.MOVEMENT like '%B%'
UNION ALL SELECT Takes.Movement FROM Takes WHERE Takes.MOVEMENT like '%B%'
UNION ALL SELECT TakesPromotion.Movement FROM TakesPromotion WHERE TakesPromotion.MOVEMENT like '%B%'
UNION ALL SELECT Promotion.Movement FROM Promotion WHERE Promotion.MOVEMENT like '%B%';

CREATE VIEW PawnMate AS
SELECT PawnMove.Movement FROM PawnMove
UNION ALL SELECT PawnTakes.Movement FROM PawnTakes;

In [109]:
DROP TABLE IF EXISTS PecasMates;

CREATE VIEW PecasMates AS
    SELECT 'Queen' TABELA,  COUNT(*) Total FROM QueenMate 
    UNION SELECT 'Rook' TABELA,  COUNT(*) Total FROM RookMate
    UNION SELECT 'King' TABELA,  COUNT(*) Total FROM KingMate
    UNION SELECT 'Bishop' TABELA,  COUNT(*) Total FROM BishopMate
    UNION SELECT 'Pawn' TABELA,  COUNT(*) Total FROM PawnMate
    UNION SELECT 'Knight' TABELA,  COUNT(*) Total FROM KnightMate
    UNION SELECT 'Total' TABELA,  COUNT(*) Total FROM Mated;

SELECT *
FROM PecasMates
ORDER BY Total DESC;